In [9]:
# https://arize.com/blog-course/react-agent-llm/

In [3]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

In [1]:
import os
os.environ["OPENAI_API_KEY"] = "..." # https://platform.openai.com/

In [10]:
# !pip install langchain-experimental

In [26]:
# llm = OpenAI(temperature=0) #Temp = 0 generally means less hallucinations and a more deterministic model
# https://platform.openai.com/docs/models/model-endpoint-compatibility
# llm = OpenAI(model="gpt-3.5-turbo-instruct") # /v1/completions (Legacy)
# llm = OpenAI(model="gpt-4-1106-preview", temperature=0)
llm = OpenAI(model="gpt-3.5-turbo-0301", temperature=0)

#Tools are functions that agents can use to interact with the world. These tools can be generic utilities (e.g. search), other chains, or even other agents.
tools = load_tools(["terminal"], llm=llm) #The tools the agent will have access to are: [Google search, math, wikipedia, terminal]

In [27]:
llm.model_name

'gpt-3.5-turbo-0301'

In [46]:
"""agent = initialize_agent(tools,
                         llm,
                         agent="zero-shot-react-description",
                         verbose=True)"""
agent = initialize_agent(tools,
                         llm,
                         agent="conversational-react-description",
                         verbose=True)

In [47]:
tools[0].name, tools[0].description

('terminal', 'Run shell commands on this Linux machine.')

In [48]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, SerpAPIWrapper, LLMChain

prefix = """Assist user the best you can. You have access to the following optional tools:"""
suffix = """Begin! 

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
   tools, prefix=prefix, suffix=suffix, input_variables=["input", "agent_scratchpad"]
)
print(prompt.template)

Assist user the best you can. You have access to the following optional tools:

terminal: Run shell commands on this Linux machine.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [terminal]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! 

Question: {input}
{agent_scratchpad}


In [49]:
import warnings

# Suppress UserWarnings
warnings.filterwarnings('ignore', category=UserWarning)

In [50]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)


tool_names = [tool.name for tool in tools]


agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)


agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)


agent_executor.run("Check the date and the list of the current directory")



> Entering new AgentExecutor chain...
Thought: I should use the terminal to run the 'date' and 'ls' commands
Action: terminal
Action Input: date
Observation: Sat Jan  6 04:33:23 PM +04 2024

Thought: I should also check the current directory
Action: terminal
Action Input: ls
Observation: app.py
Dockerfile
react.ipynb
requirements.txt
terminal_fake.ipynb
terminal_native.ipynb
tools.ipynb

Thought: I now know the final answer
Final Answer: The date is Sat Jan  6 04:33:23 PM +04 2024 and the current directory contains the files app.py, Dockerfile, react.ipynb, requirements.txt, terminal_fake.ipynb, terminal_native.ipynb, and tools.ipynb.

> Finished chain.


'The date is Sat Jan  6 04:33:23 PM +04 2024 and the current directory contains the files app.py, Dockerfile, react.ipynb, requirements.txt, terminal_fake.ipynb, terminal_native.ipynb, and tools.ipynb.'

In [51]:
agent_executor.run("My name is Bob. What is your name?")



> Entering new AgentExecutor chain...
Thought: I should introduce myself and ask for the user's name.
Action: terminal
Action Input: whoami
Observation: alex

Thought: I should greet the user and ask for their name.
Action: terminal
Action Input: echo "Hello, what is your name?"
Observation: /bin/sh: 1: Syntax error: Unterminated quoted string

Thought: I should use the read command to get the user's input.
Action: terminal
Action Input: read name
Observation: 
Thought: I should greet the user by their name.
Action: terminal
Action Input: echo "Hello $name, my name is Alex."
Observation: /bin/sh: 1: Syntax error: Unterminated quoted string

Thought: I should ask the user how I can assist them.
Action: terminal
Action Input: echo "How can I assist you?"
Observation: /bin/sh: 1: Syntax error: Unterminated quoted string

Thought: I should wait for the user's input.
Action: terminal
Action Input: read question
Observation: 
Thought: I should provide an answer based on the user's input.
A

"I'm sorry, I am just a simple AI and I am not able to assist with that question."

In [52]:
agent_executor.run("Do you remember my name? Call my name please")



> Entering new AgentExecutor chain...
Thought: I should check if I have access to the user's name
Action: terminal
Action Input: whoami
Observation: alex

Thought: I can use the user's name to call them
Action: terminal
Action Input: echo "Hello, alex!"
Observation: /bin/sh: 1: Syntax error: Unterminated quoted string

Thought: I need to use double quotes to properly format the string
Action: terminal
Action Input: echo "Hello, alex!"
Observation: /bin/sh: 1: Syntax error: Unterminated quoted string

Thought: I should try using single quotes instead
Action: terminal
Action Input: echo 'Hello, alex!'
Observation: Hello, alex!

Thought: I have successfully called the user by their name
Final Answer: Hello, alex!

> Finished chain.


'Hello, alex!'